# Импортируем библиотеки

In [63]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split

import lightgbm as lgbm
import xgboost as xgb
import catboost

# Загружаем датасеты

In [76]:
# Загружаем данные

train_data = pd.read_csv('data/train_processed.csv')
valid_data = pd.read_csv('data/valid_processed.csv')

import re
train_data = train_data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
valid_data = valid_data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# Подготавливаем данные для обучения моделей

In [77]:
# Разделяем датасеты на выборки

# Разделяем train датасет на таргет и признаки
X_train = train_data.drop(columns=['target'])  # Признаки
y_train = train_data['target']  # Таргет

# Разделяем valid датасет на таргет и признаки
X_valid = valid_data.drop(columns=['target']) # Признаки
y_valid = valid_data['target'] # Таргет

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Функция вывода метрик модели

In [78]:
def print_metrics(model):
    # Оцениваем LightGBM модель
    # test выборка
    test_predictions = model.predict(X_test)
    test_probabilities = model.predict_proba(X_test)[:, 1]

    test_accuracy = accuracy_score(y_test, test_predictions)
    test_presicion = precision_score(y_test, test_predictions)
    test_recall = recall_score(y_test, test_predictions)
    test_f1score = f1_score(y_test, test_predictions)
    test_roc_auc = roc_auc_score(y_test, test_probabilities)

    # valid выборка
    valid_predictions = model.predict(X_valid)
    valid_probabilities = model.predict_proba(X_valid)[:, 1]

    valid_accuracy = accuracy_score(y_valid, valid_predictions)
    valid_presicion = precision_score(y_valid, valid_predictions)
    valid_recall = recall_score(y_valid, valid_predictions)
    valid_f1score = f1_score(y_valid, valid_predictions)
    valid_roc_auc = roc_auc_score(y_valid, valid_probabilities)

    # Выводим метрики
    print(f"""\nТест на тестовой выборке
    Accuracy: {test_accuracy:.4f}
    Precision: {test_presicion:.4f}
    Recall: {test_recall:.4f}
    F1-Score: {test_f1score:.4f}
    ROC-AUC: {test_roc_auc:.4f}\n
    """)

    print(f"""Тест на валидационной выборке
    Accuracy: {valid_accuracy:.4f}
    Precision: {valid_presicion:.4f}
    Recall: {valid_recall:.4f}
    F1-Score: {valid_f1score:.4f}
    ROC-AUC: {valid_roc_auc:.4f}
    """)
    return

# LightGBM

In [79]:
# Обучаем модель
lgbm_model = lgbm.LGBMClassifier()
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 353, number of negative: 11211
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012646 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13955
[LightGBM] [Info] Number of data points in the train set: 11564, number of used features: 150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.030526 -> initscore=-3.458183
[LightGBM] [Info] Start training from score -3.458183


LGBMClassifier()

In [80]:
# Выводим метрики по LightGBM
print_metrics(lgbm_model)


Тест на тестовой выборке
    Accuracy: 0.9837
    Precision: 0.8000
    Recall: 0.5783
    F1-Score: 0.6713
    ROC-AUC: 0.9754

    
Тест на валидационной выборке
    Accuracy: 0.9822
    Precision: 0.6000
    Recall: 0.0455
    F1-Score: 0.0845
    ROC-AUC: 0.7210
    


# XGBoost

In [81]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [83]:
print_metrics(xgb_model)


Тест на тестовой выборке
    Accuracy: 0.9848
    Precision: 0.7910
    Recall: 0.6386
    F1-Score: 0.7067
    ROC-AUC: 0.9654

    
Тест на валидационной выборке
    Accuracy: 0.9698
    Precision: 0.1667
    Recall: 0.1667
    F1-Score: 0.1667
    ROC-AUC: 0.6857
    


# CatBoost

In [84]:
catboost_model = catboost.CatBoostClassifier()



catboost_model.fit(X_train, y_valid)

CatBoostError: Length of label=3642 and length of data=11564 is different.